In [1]:
import gradio as gr
import importlib
import os
from IPython.display import Markdown
from python_scripts import llm_rag, machine_translation, text_to_speech, whisper_setup, get_audio, utils
import numpy as np
from scipy.io.wavfile import write
# r8_eYbLLc9wPQ1qa4dUK8eXyewFKHvGAck1h9LZl

# Define paths dynamically
PATH = os.getcwd().replace('\\\\', '/')
root_path = PATH + '\\Datasets\\MeDAL'
audio_path = PATH + '\\Datasets\\Audio_Files'

In [2]:

importlib.reload(llm_rag)
# importlib.reload(machine_translation)
# importlib.reload(text_to_speech)
# importlib.reload(whisper_setup)
# importlib.reload(get_audio)
# importlib.reload(utils)


<module 'python_scripts.llm_rag' from 'c:\\Users\\Admin\\OneDrive\\Documents\\GitHub\\CS-5302-Project-Group-15\\python_scripts\\llm_rag.py'>

In [2]:
def SMTS(audio):
    try:

        file_path = 'output.wav'
        write(file_path, data = np.array(audio[1], dtype = np.int16), rate = audio[0])
        audio_processed = utils.preprocess_audio(file_path)

        # Step 2: Transcribe Query to English
        whisper_models = ["tiny", "base", "small", "medium", "large"]
        
        transcript = whisper_setup.transcribe_audio(audio_processed, ['tiny'])

        # Step 3: Feed query into the LLM
        models = { 
        'llama_ours': 'ubaidtariq8/llama2-med-genai', # fine tuned model
        'llama_13b': 'a16z-infra/llama13b-v2-chat:df7690f1994d94e96ad9d568eac121aecf50684a0b0963b25a41cc40061269e5',
        'mixtral': 'mistralai/mixtral-8x7b-instruct-v0.1',
        'llama_70b': 'meta/llama-2-70b-chat:2796ee9483c3fd7aa2e171d38f4ca12251a30609463dcfd4cd76703f22e96cdf'
        }

        model = llm_rag.DocumentEmbeddingPipeline(model_version = models['mixtral'], chroma_path = root_path)
        model.setup_environment()
        model.prepare_documents(collection_name = "muqeem", joining = True, persistent = True)
        model.embed_and_index()
        response = model.query_data(query = transcript['tiny'][2])

        # # Step 4: Translate it back to the user language
        translated_text = machine_translation.translate_text(text = response.response, src_lang = 'en', trg_lang = transcript['tiny'][0])
        print(translated_text)
        # Step 5: Now speak the response in the user's language
        audio_answer_path = audio_path + '/audio.wav'
        text_to_speech.multilingual_text_to_speech(text = translated_text, filepath = audio_answer_path)
        utils.sasti_harkat(audio_answer_path)
        # display(Markdown(f"<b>{translated_text}</b>"))
        return transcript['tiny'][2], utils.play_wav(audio_answer_path)
    except Exception as e:
        print("An error occurred:", e)

In [3]:
demo = gr.Interface(
    SMTS,
    gr.Audio(sources = ["microphone"]),
    ["text", "audio"],
)

demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Loading files: 100%|██████████| 2/2 [00:00<00:00, 28.48file/s]


Parsing nodes:   0%|          | 0/2 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/71 [00:00<?, ?it/s]

HELLO!
Based on the information provided, it is possible that you may have influenza. However, please note that this is not a definitive diagnosis. It is crucial to consult with a healthcare professional for an accurate diagnosis and treatment.
Based on the information provided, it is possible that you may have influenza. However, please note that this is not a definitive diagnosis. It is crucial to consult with a healthcare professional for an accurate diagnosis and treatment.
!!
Detected language: en
Speech saved to c:\Users\Talha\OneDrive - Higher Education Commission\Documents\GitHub\CS-5302-Project-Group-15\Datasets\Audio_Files/audio.wav
gg
FGG
